In [10]:
import requests
import pandas as pd
import numpy as np
import io
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playergamelog, teamgamelog
import matplotlib.pyplot as plt
from datetime import datetime

In [48]:
player_dict = players.get_players()
jalen = [player for player in player_dict if player['full_name'] == 'Jalen Green'][0]
jalen_id = jalen['id']

gamelog_jalen = playergamelog.PlayerGameLog(player_id=jalen_id, season='2021')

df_jalen_games_2021 = gamelog_jalen.get_data_frames()[0]

In [33]:
def get_datetime(time):
    datetime_object = datetime.strptime(time, '%b %d, %Y')
    return datetime_object

In [49]:
df_jalen_games_2021['GAME_DATE'] = df_jalen_games_2021['GAME_DATE'].apply(get_datetime)
df_jalen_games_2021.loc[df_jalen_games_2021['GAME_DATE'] > get_datetime('FEB 20, 2022'), 'POST_ALL_STAR'] = 'POST ALL STAR'
df_jalen_games_2021.loc[df_jalen_games_2021['GAME_DATE'] < get_datetime('FEB 20, 2022'), 'POST_ALL_STAR'] = 'PRE ALL STAR'

In [50]:

print(df_jalen_games_2021.columns)
df_jalen_games_2021.head(5)

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'POST_ALL_STAR'],
      dtype='object')


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,POST_ALL_STAR
0,22021,1630224,0022100964,2022-03-06,HOU vs. MEM,W,35,8,17,0.471,...,8,4,0,0,2,2,24,7,1,POST ALL STAR
1,22021,1630224,0022100952,2022-03-04,HOU @ DEN,L,32,7,14,0.500,...,5,7,0,1,1,1,18,-8,1,POST ALL STAR
2,22021,1630224,0022100935,2022-03-02,HOU vs. UTA,L,47,11,16,0.688,...,2,4,1,1,4,1,27,4,1,POST ALL STAR
3,22021,1630224,0022100929,2022-03-01,HOU vs. LAC,L,33,9,21,0.429,...,4,1,2,1,0,1,20,-9,1,POST ALL STAR
4,22021,1630224,0022100916,2022-02-27,HOU vs. LAC,L,38,4,18,0.222,...,3,3,2,0,0,1,10,4,1,POST ALL STAR


In [51]:
df_jalen_games_2021.groupby('POST_ALL_STAR')['FG3_PCT'].mean()

POST_ALL_STAR
POST ALL STAR    0.312333
PRE ALL STAR     0.289721
Name: FG3_PCT, dtype: float64

In [58]:
df_jalen_agg = df_jalen_games_2021.groupby('POST_ALL_STAR').agg(
    
    GAMES = ('GAME_DATE', "count"),
    PPG = ('PTS', "mean"),
    APG = ('AST', "mean"),
    REB = ('REB', "mean"),
    FG_PCT = ('FG_PCT', "mean"),
    FG3_PCT = ('FG3_PCT', "mean"),
    FGAPG = ('FGA', "mean")

)

In [59]:
df_jalen_agg

,GAMES,PPG,APG,REB,FG_PCT,FG3_PCT,FGAPG
POST_ALL_STAR,,,,,,,
POST ALL STAR,6,20.333333,3.500000,4.500000,0.478833,0.312333,17.00000
PRE ALL STAR,43,14.581395,2.325581,3.116279,0.389488,0.289721,12.55814
